# Verify Space Motion Functionality

In [22]:
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord

#exagerated motions for testing purposes
c = SkyCoord(l=10*u.degree, b=45*u.degree, distance=100*u.pc,
             pm_l_cosb=340*u.mas/u.yr, pm_b=-1170*u.mas/u.yr,
             frame='galactic',
             obstime=Time('1988-12-18 05:11:23.5'))

In [23]:
c

<SkyCoord (Galactic): (l, b, distance) in (deg, deg, pc)
    (10., 45., 100.)
 (pm_l_cosb, pm_b) in mas / yr
    (340., -1170.)>

In [24]:
dt = 10*u.year
c_new = c.apply_space_motion(dt=dt)
c_new

<SkyCoord (Galactic): (l, b, distance) in (deg, deg, pc)
    (10.00133557, 44.99674999, 99.99999448)
 (pm_l_cosb, pm_b, radial_velocity) in (mas / yr, mas / yr, km / s)
    (339.98071452, -1170.00559973, 0.03411744)>

In [27]:
import numpy as np
np.allclose(c_new.b, (c.b + c.pm_b*dt))

True

In [31]:
f'{c_new.b/ (c.b + c.pm_b*dt):.10f}'

'0.9999999998'

In [21]:
np.allclose(c_new.l, (c.l+(c.pm_l_cosb/np.cos(c.b))*dt))

True

`apply_space_motion` appears to work the way I think it should, that is:
\begin{equation}
\begin{align*}
\delta' &= \delta + \dot{\delta}dt \cr
\alpha' &= \alpha + \dot{\alpha}dt
\end{align*}
\end{equation}

# Verify Time Functionality

In [32]:
from astropy.time import Time
import astropy.units as u

# gaia ref_epoch on each gaia source record:
ref_epoch = 2016 #for dr3
# gaia doc specifies ref_epoch is tcb
t_gaia = Time(ref_epoch, scale='tcb',format='jyear')
t_gaia

<Time object: scale='tcb' format='jyear' value=2016.0>

In [33]:
t_gaia.iso

'2016-01-01 12:00:00.000'

In [34]:
#Fits header has OBSTIME as MJD
mjd = 53739.304355 # value from the header
t_obs = Time(mjd, scale='utc', format='mjd')
t_obs

<Time object: scale='utc' format='mjd' value=53739.304355>

In [35]:
t_obs.iso

'2006-01-04 07:18:16.272'

In [36]:
# calculate difference btwn the two times
#difference in seconds
(t_obs-t_gaia).sec


-315290419.4613611

In [37]:
#difference in years
(((t_gaia-t_obs).sec)*u.second).to(u.year)

<Quantity 9.99095065 yr>

In [38]:
c = SkyCoord(l=10*u.degree, b=45*u.degree, distance=100*u.pc,
             pm_l_cosb=340*u.mas/u.yr, pm_b=-1170*u.mas/u.yr,
             frame='galactic',
             obstime=t_gaia)
c_new = c.apply_space_motion(t_obs)

In [40]:
dt = t_obs-t_gaia
np.allclose(c_new.l, (c.l+(c.pm_l_cosb/np.cos(c.b))*dt))

True

In [41]:
np.allclose(c_new.b, (c.b + c.pm_b*dt))

True

Time and motions seem to check out!!

In [108]:
from gaiastars import gaiastars as gs
def gaia_from_image(hdu):
    hdr = dict(hdu.header)
    gstars = gs(name = hdr['OBJECT'], description =hdr['S_UFNAME'])

    ra = hdr['CRVAL1']*u.degree
    dec = hdr['CRVAL2'] * u.degree
    radius = abs(hdr['CDELT1']*hdr['CRPIX1']) * u.degree

    gstars.conesearch(ra, dec, radius)

    return gstars

In [120]:
from astropy.time import Time
import astropy.coordinates as coord

def __get_coords__(self, recalc, default_rv):
    #computes and caches sky coordinates for the objects
    #set recalc=True to force recalculation

    if self.coords is None or recalc:
        if default_rv is None:
            rv = None
        elif isinstance(default_rv, bool):
            rv = np.array(self.objs.radial_velocity)*u.km/u.s if default_rv else None
        else:
            #use the reported rv if available otherwise the default rv
            rv = np.where(np.isfinite(self.objs.radial_velocity),
                            self.objs.radial_velocity,
                            default_rv)*u.km/u.s
        #hard code for gaia dr3:
        t_gaia = Time(2016, scale='tcb',format='jyear')

        self.coords = coord.SkyCoord(ra=np.array(self.objs.ra)*u.degree,
                dec=np.array(self.objs.dec)*u.degree,
                distance=np.array(self.objs.r_est)*u.pc,
                pm_ra_cosdec=np.array(self.objs.pmra)*u.mas/u.yr,
                pm_dec=np.array(self.objs.pmdec)*u.mas/u.yr,
                radial_velocity=rv,
                obstime = t_gaia)

def get_coords(self, recalc=False, default_rv = None, newobstime=None):
    #returns sky coordinates for the objects
    self.__get_coords__(
            recalc, default_rv)
    if newobstime is not None:
        return self.coords.apply_space_motion(newobstime)
    else:
        return self.coords

gs.__get_coords__=__get_coords__
gs.get_coords = get_coords